# `log_*` methods

In minto, you can record data using methods called `log_*`.

This section introduces basic methods and some convenient methods.

## Basic `log` methods

In `minto`, there are two spaces, and data is stored in the form of a class called `DataStore` in each space. For details, see the [MINTO Spaces](minto_datastore.ipynb) section.

To save to the Experiment space, use

```
experiment.log_global_*
```
methods, and to save to the Run space, use

```
run.log_*
```

methods. In place of `*`, you can put names corresponding to `DataStore` attributes.

Here is a list of basic `log` methods:

|data type|Experiment space | Run space|
|--|--|--|
|scalars (int \| float \| str) | log_global_parameter | log_parameter|
|`jm.Problem` | log_global_problem | log_problem|
|`ommx.v1.Instance` | log_global_instance | log_instance|
|`ommx.v1.Solution` | log_global_solution | log_solution|
|`ommx.v1.SampleSet` | log_global_sampleset | log_sampleset|
|JSON Serializable object (dict)| log_global_object | log_object |

## params

When you want to save multiple scalars, you can use the `log_params` method to save multiple at once as a dict.

In [ ]:
import minto

experiment = minto.Experiment("test", auto_saving=False, verbose_logging=False)

param_a = [1, 2, 3, 4]
param_b = [2, 3, 4, 5]

for a, b in zip(param_a, param_b):
    with experiment.run() as run:
        run.log_params({
            "a": a,
            "b": b
        })

In [ ]:
experiment.get_run_table()

You can also save multiple parameters at once in the `Experiment` space.

In [ ]:
experiment.log_global_params({"a": a, "b": b})

experiment.get_experiment_tables()["parameter"]

# `solver`

It is possible to wrap an optimization function and save its inputs and outputs as is.

`log_solver` returns a `Callable` object that behaves the same as the passed function. When you call that object, the original function is executed, and its inputs and outputs are saved to the Run space. You can also pass a list of argument names you don't want to save to the run in the `exclude_params` of `log_solver`. This is convenient when you don't want to save large data like instances in the run.

Rewriting the example introduced in the [Quick start](quickstart.ipynb) section using `log_solver` would look like this:

In [ ]:
import ommx_pyscipopt_adapter as scip_ad
from ommx.dataset import miplib2017

instance = miplib2017("reblock115")

def scip_solver(instance, time_limit):
    adapter = scip_ad.OMMXPySCIPOptAdapter(instance)
    scip_model = adapter.solver_input

    scip_model.setParam("limits/time", time_limit)
    scip_model.optimize()

    return adapter.decode(scip_model)

time_limit_list = [0.5, 1.0]

experiment = minto.Experiment("test", auto_saving=False, verbose_logging=False)

for time_limit in time_limit_list:
    with experiment.run() as run:
        _solver = run.log_solver(scip_solver, exclude_params=["instance"])
        solution = _solver(instance, time_limit=time_limit)

In [ ]:
experiment.get_run_table()

## Summary

By using `log_*` methods, you can easily save various data to the Experiment space and Run space.
`params` and `solver` can be used as convenient wrappers, so please make use of them.